# Introduction

### Feature Slection vs Feature Extraction 

Dimensionality Reduction에는 2가지 종류의 스킬이 있습니다. 

1. Feature Selection: original features의 일부분 (subset)을 선택합니다. 
2. Feature Extraction: original features에서 중요한 정보를 꺼내서 **새로운** feature subspace를 만듭니다.

### Sequential Feature Selection Algorithm

Sequential Feature Selection Algorithm은 greedy search algorithm으로서 <br>
d-dimensional feature space를 k-dimensional feature subspace로 바꿔줍니다. <br>
이때 k < d 라는 조건을 갖습니다.

**특히 regularization을 지원하지 않는 알고리즘에 사용이 될 수 있는 장점이 있습니다. **

특히 Model이 overfitting을 겪을 경우, Sequential Backward Selection (SBS)같은 알고리즘이.. <br>
간혹 prediction수치 자체를 높이는 경우도 있습니다.

# Sequential Backward Selection (SBS)

SBS알고리즘은 지정한 갯수의 features들을 뽑을때까지 계속 feature를 지워나가는 형태입니다. <br>
이를 위해서 **Criterion function J**를 정의해야 합니다. 

$$  $$